Install libraries

In [2]:
!pip install sentinelsat

Import libraries

In [0]:
from sentinelsat import SentinelAPI

Read login credentials

In [0]:
with open('/content/login_info.txt') as file:
  credentials = file.readlines()
  credentials = [credential.strip() for credential in credentials] 

In [0]:
api = SentinelAPI(credentials[0], credentials[1], 'https://scihub.copernicus.eu/dhus')